In [1]:
%run ../scripts/notebook_settings.py
import sgkit as sg
import xarray as xr
import glob

from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

In [2]:
vasili_stats = "~/primatediversity/data/gVCFs_recalling_10_12_2024_metadata/plots/SupTable_Sample_Stats_wGT_QC.tsv"
metadata_path = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024_metadata/"

vasili_table = pd.read_csv(vasili_stats, sep="\t")
sub_vasili = vasili_table.loc[(vasili_table.finalQC != "fail")
                              & (vasili_table.cov_chrA >= 20)
                              & (vasili_table.remove_as_relative != True)
                              & (vasili_table.remove_manual != True)
                              & (~vasili_table.ID.str.startswith("SAMEA11633"))]


In [3]:
female_IDs = sub_vasili.loc[sub_vasili.gSEX == "F"].ID

Here, I will pick out the best cluster based on number of X haplotypes (1 for males, 2 for females) and divergence from reference. If possible, I want 3 or more X haplotypes.

In [4]:
#glob.glob("../results/window_stats_20x_inds/*0.1cutoff_fst*")[:1]
for s in sub_vasili.species_genotyping.unique():
    print(s)

Allenopithecus_nigroviridis_ssp
Aotus_azarai_ssp
Aotus_griseimembra_ssp
Aotus_nancymaae_ssp
Aotus_trivirgatus_ssp
Aotus_vociferans_ssp
Alouatta_belzebul_ssp
Alouatta_caraya_ssp
Alouatta_discolor_ssp
Alouatta_juara_ssp
Alouatta_macconnelli_ssp
Alouatta_palliata_ssp
Alouatta_seniculus_ssp
Ateles_belzebuth_ssp
Ateles_chamek_ssp
Ateles_geoffroyi_ssp
Ateles_marginatus_ssp
Ateles_paniscus_ssp
Brachyteles_hypoxanthus_ssp
Lagothrix_lagotricha_ssp
Callimico_goeldii_ssp
Callithrix_geoffroyi_ssp
Callithrix_jacchus_ssp
Callithrix_kuhlii_ssp
Cebuella_niveiventris_ssp
Cebuella_pygmaea_ssp
Mico_argentatus_ssp
Mico_humeralifer_ssp
Mico_humilis_ssp
Carlito_syrichta_ssp
Cephalopachus_bancanus_ssp
Tarsius_dentatus_ssp
Tarsius_lariang_ssp
Tarsius_wallacei_ssp
Cercopithecus_ascanius_ssp
Cercopithecus_campbelli_ssp
Cercopithecus_cephus_ssp
Cercopithecus_denti_ssp
Cercopithecus_diana_ssp
Cercopithecus_hamlyni_ssp
Cercopithecus_mitis_ssp
Cercopithecus_mona_ssp
Cercopithecus_neglectus_ssp
Cercopithecus_nictita

In [5]:
len(sub_vasili.species_genotyping.unique()), len(glob.glob("../results/window_stats_20x_inds/*100kb_pi*"))

(251, 3)

In [15]:
glob.glob("../results/window_stats_20x_inds/*100kb_pi*")

['../results/window_stats_20x_inds/Cercopithecus_campbelli_ssp_100kb_pi_fst0.1.txt',
 '../results/window_stats_20x_inds/Pygathrix_nigripes_ssp_100kb_pi_fst0.1.txt']

Investigation of why there are some species with large numbers of NaN windows.

In [14]:
df = pd.read_csv('../results/window_stats_20x_inds/Cheirogaleus_medius_ssp_100kb_pi_fst0.1.txt', sep="\t")
df.loc[(df.chr_type == "chrX") & (df.pi_0 > 0)]#.chr_type.unique()

,window_start,chrom,pi_0,divergence_0,window_end,callable_frac,chr_type,species
2512,0,VBSM01001483.1,42.933333,34.666667,100000,0.87654,chrX,Cheirogaleus_medius_ssp
2513,100000,VBSM01001483.1,15.933333,11.333333,200000,0.92534,chrX,Cheirogaleus_medius_ssp
2514,200000,VBSM01001483.1,22.666667,19.666667,300000,0.93875,chrX,Cheirogaleus_medius_ssp
2515,300000,VBSM01001483.1,47.033333,35.000000,400000,0.90058,chrX,Cheirogaleus_medius_ssp
2516,400000,VBSM01001483.1,37.666667,36.666667,500000,0.92065,chrX,Cheirogaleus_medius_ssp
...,...,...,...,...,...,...,...,...
20585,4500000,VBSM01000937.1,21.466667,21.666667,4600000,0.95388,chrX,Cheirogaleus_medius_ssp
20586,4600000,VBSM01000937.1,44.400000,34.000000,4700000,0.96104,chrX,Cheirogaleus_medius_ssp
20587,4700000,VBSM01000937.1,12.533333,14.333333,4800000,0.95822,chrX,Cheirogaleus_medius_ssp
20588,4800000,VBSM01000937.1,24.800000,26.000000,4900000,0.97115,chrX,Cheirogaleus_medius_ssp


In [25]:
sub_vasili.loc[sub_vasili.species_genotyping == "Cheracebus_regulus_ssp"]

,ID,ID_PGDP,group,family,genus,species,subspecies,species_full,species_short,species_genotyping,...,reason,finalQC,Kuderna_AB_QC,Kuderna_COV_QC,relatives,remove_as_relative,generation,founder,remove_manual,pedigree_source
2763,PD_0284,PD_0284,PLA,Pitheciidae,Cheracebus,regulus,ssp,Cheracebus_regulus_ssp,C.regulus_ssp,Cheracebus_regulus_ssp,...,NaN,ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2764,PD_0322,PD_0322,PLA,Pitheciidae,Cheracebus,regulus,ssp,Cheracebus_regulus_ssp,C.regulus_ssp,Cheracebus_regulus_ssp,...,NaN,ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, labels=aut_fst.columns, orientation="left", **kwargs)

Looking through the various fst clusters.

In [ ]:
for p in glob.glob("../results/window_stats_20x_inds/*0.1cutoff_fst*"):
    aut_fst = pd.read_csv(p, sep="\t")
    model = AgglomerativeClustering(distance_threshold=0.1, n_clusters=None, metric="precomputed", linkage="average")
    model = model.fit(aut_fst.clip(0))
    plt.title("{} Dendrogram".format(p.split("/")[-1].split("_ssp_")[0]))
    plot_dendrogram(model, truncate_mode="level", p=8)
    plt.figure()

In [ ]:
for p in glob.glob("../results/window_stats_20x_inds/Ma*0.1cutoff_fst*"):
    aut_fst = pd.read_csv(p, sep="\t")
    model = AgglomerativeClustering(distance_threshold=0.1, n_clusters=None, metric="precomputed", linkage="average")
    model = model.fit(aut_fst.clip(0))
    plt.title("{} Dendrogram".format(p.split("/")[-1].split("_ssp_")[0]))
    plot_dendrogram(model, truncate_mode="level", p=8)
    plt.figure()

In [ ]:
len(glob.glob("../results/window_stats_20x_inds/*0.1cutoff_fst*"))

In [ ]:
len(glob.glob("../results/window_stats_20x_inds/*100kb_pi*"))